# Exercises

This is the notebook containing the exercises for Feature Store, Model Monitor, and Clarify. Tested for these exercises was performed using __2 vCPU + 4 GiB notebook instance with Python 3 (TensorFlow 2.1 Python 3.6 CPU Optimized) kernel__.

## Staging

We'll begin by initializing some variables. These are often assumed to be present in code samples you'll find in the AWS documenation.

In [1]:
import sagemaker
from sagemaker.session import Session
from sagemaker import get_execution_role

role = get_execution_role()
session = sagemaker.Session()
region = session.boto_region_name
bucket = session.default_bucket()

Unable to load JumpStart region config.
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.12/site-packages/sagemaker/jumpstart/constants.py", line 69, in _load_region_config
    with open(filepath) as f:
         ^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.12/site-packages/sagemaker/jumpstart/region_config.json'


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## Feature Store
---

Feature store is a special database to give ML systems a consistent data flow across training and inference workloads. It can ingest data in batches (for training) as well as serve input features to models with very low latency for real-time prediction.

For this exercise we'll work with a wine quality dataset: https://archive.ics.uci.edu/ml/datasets/wine+quality/

```P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.```

In [2]:
import pandas as pd
from sklearn import datasets
import time
import uuid

In [3]:
data = datasets.load_wine()
df = pd.DataFrame(data['data'])
df.columns = data['feature_names']

If we leave the column names as-is, Feature Store won't be able to handle the `/` in `od280/od315_of_diluted_wines` (`/` is a delimiter Feature Store uses to manage how features are organized.)

In [4]:
df.rename(columns = {'od280/od315_of_diluted_wines':'od280_od315_of_diluted_wines'}, inplace=True)

Once we have our data, we can create a feature group. Remember to attach event time and ID columns - Feature Store needs them.

In [10]:
# Add a feature group
df["EventTime"] = time.time()
df["id"] = range(len(df))

# Create feature group
from sagemaker.feature_store.feature_group import FeatureGroup

feature_group = FeatureGroup(
    name="wine_quality", sagemaker_session=session
)

# Load Feature definitions
feature_group.load_feature_definitions(data_frame=df)

[FeatureDefinition(feature_name='alcohol', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='malic_acid', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='ash', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='alcalinity_of_ash', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='magnesium', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='total_phenols', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='flavanoids', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='nonflavanoid_phenols', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collec

The feature group is not created until we call the `create` method, let's do that now:

# Create the feature store:

In [11]:
# create feature store
feature_group.create(
    s3_uri=f"s3://{bucket}/features",
    record_identifier_name = 'id',
    event_time_feature_name = "EventTime",
    role_arn=role,
    enable_online_store=True,
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # create feature store                                                                       │
│ ❱ 2 feature_group.create(                                                                        │
│   3 │   s3_uri=f"s3://{bucket}/features",                                                        │
│   4 │   record_identifier_name = 'id',                                                           │
│   5 │   event_time_feature_name = "EventTime",                                                   │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.12/site-packages/sagemaker/feature_st │
│ ore/feature_group.py:789 in create                                                               │
│                                                                                                  │
│    786 │   │   │   │   {"offline_store_config": offline_store_config.to_dict()}                  │
│    787 │   │   │   )                                                                             │
│    788 │   │                                                                                     │
│ ❱  789 │   │   return self.sagemaker_session.create_feature_group(**create_feature_store_args)   │
│    790 │                                                                                         │
│    791 │   def delete(self):                                                                     │
│    792 │   │   """Delete a FeatureGroup."""                                                      │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.12/site-packages/sagemaker/session.py │
│ :6614 in create_feature_group                                                                    │
│                                                                                                  │
│   6611 │   │   │   Description=description,                                                      │
│   6612 │   │   │   Tags=tags,                                                                    │
│   6613 │   │   )                                                                                 │
│ ❱ 6614 │   │   return self.sagemaker_client.create_feature_group(**kwargs)                       │
│   6615 │                                                                                         │
│   6616 │   def describe_feature_group(                                                           │
│   6617 │   │   self,                                                                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.12/site-packages/botocore/client.py:6 │
│ 02 in _api_call                                                                                  │
│                                                                                                  │
│    599 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    600 │   │   │   │   )                                                                         │
│    601 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  602 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    603 │   │                                                                                     │
│    604 │   │   _api_call.__name__ = str(py_operation_name) 

Lastly, ingest some data into your feature group:

In [12]:
# ingest data into feature group
feature_group.ingest(data_frame=df, max_workers=3, wait=True)

IngestionManagerPandas(feature_group_name='wine_quality', feature_definitions={'alcohol': {'FeatureName': 'alcohol', 'FeatureType': 'Fractional'}, 'malic_acid': {'FeatureName': 'malic_acid', 'FeatureType': 'Fractional'}, 'ash': {'FeatureName': 'ash', 'FeatureType': 'Fractional'}, 'alcalinity_of_ash': {'FeatureName': 'alcalinity_of_ash', 'FeatureType': 'Fractional'}, 'magnesium': {'FeatureName': 'magnesium', 'FeatureType': 'Fractional'}, 'total_phenols': {'FeatureName': 'total_phenols', 'FeatureType': 'Fractional'}, 'flavanoids': {'FeatureName': 'flavanoids', 'FeatureType': 'Fractional'}, 'nonflavanoid_phenols': {'FeatureName': 'nonflavanoid_phenols', 'FeatureType': 'Fractional'}, 'proanthocyanins': {'FeatureName': 'proanthocyanins', 'FeatureType': 'Fractional'}, 'color_intensity': {'FeatureName': 'color_intensity', 'FeatureType': 'Fractional'}, 'hue': {'FeatureName': 'hue', 'FeatureType': 'Fractional'}, 'od280_od315_of_diluted_wines': {'FeatureName': 'od280_od315_of_diluted_wines', 'Fe

Great job! You've demonstrated your understanding of creating feature groups and ingesting data into them using Feature Store. Next up we'll cover Model Monitor!

## Model Monitor

In this exercise we'll create a monitoring schedule for a deployed model. We're going to provide code to help you deploy a model and get started, so that you can focus on Model Monitor for this exercise. __Remember to clean up your model before you end a work session__. We'll provide some code at the end to help you clean up your model. We'll begin by reloading our data from the previous exercise.



In [13]:
data = datasets.load_wine()
df = pd.DataFrame(data['data'])
df.columns = data['feature_names']
df.rename(columns = {'od280/od315_of_diluted_wines':'od280_od315_of_diluted_wines'}, inplace=True)

We also need to put the target variable in the first column per the docs for our chosen algorithm: https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html

In [14]:
df["TARGET"] = data['target']
df.set_index(df.pop('TARGET'), inplace=True)
df.reset_index(inplace=True)

Now we'll upload the data to S3 as train and validation data:

In [15]:
delimiter = int(len(df)/2)
train, test = df.iloc[delimiter:], df.iloc[:delimiter]

train.to_csv("train.csv", header=False, index=False)
test.to_csv("validation.csv", header=False, index=False)

val_location = session.upload_data('./validation.csv', key_prefix="data")
train_location = session.upload_data('./train.csv', key_prefix="data")

s3_input_train = sagemaker.inputs.TrainingInput(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data=val_location, content_type='csv')

In [16]:
algo_image = sagemaker.image_uris.retrieve("xgboost", region, version='latest')
s3_output_location = f"s3://{bucket}/models/wine_model"

model=sagemaker.estimator.Estimator(
    image_uri=algo_image,
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    volume_size=5,
    output_path=s3_output_location,
    sagemaker_session=sagemaker.Session()
)

model.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        objective='reg:linear',
                        early_stopping_rounds=10,
                        num_round=200)


model.fit({'train': s3_input_train, 'validation': s3_input_validation})

INFO:sagemaker:Creating training-job with name: xgboost-2026-01-25-20-59-30-301


2026-01-25 20:59:31 Starting - Starting the training job...
2026-01-25 20:59:46 Starting - Preparing the instances for training...
2026-01-25 21:00:11 Downloading - Downloading input data...
2026-01-25 21:00:41 Downloading - Downloading the training image......
2026-01-25 21:01:57 Training - Training image download completed. Training in progress.
2026-01-25 21:01:57 Uploading - Uploading generated training modelArguments: train
[2026-01-25:21:01:52:INFO] Running standalone xgboost training.
[2026-01-25:21:01:52:INFO] File size need to be processed in the node: 0.01mb. Available memory size in the node: 8529.61mb
[2026-01-25:21:01:52:INFO] Determined delimiter of CSV input is ','
[21:01:52] S3DistributionType set as FullyReplicated
[21:01:52] 89x13 matrix with 1157 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2026-01-25:21:01:52:INFO] Determined delimiter of CSV input is ','
[21:01:52] S3DistributionType set as FullyReplicated
[21:01:52] 89x13 mat

Now that your training job has finished, you can perform the first task in this exercise: creating a data capture config. Configure your model to sample `34%` of inferences:

In [17]:
# set model capture
from sagemaker.model_monitor import DataCaptureConfig

capture_uri = f's3://{bucket}/data-capture'

data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri = capture_uri
)
# Model Capture in AWS SageMaker is a monitoring feature that automatically records data 
# passing through  deployed models in production
# detect data drift and concept drift

Great! We'll use your config to deploy a model below:

In [18]:
xgb_predictor = model.deploy(
    initial_instance_count=1, instance_type='ml.m4.xlarge',
    data_capture_config=data_capture_config
)

INFO:sagemaker:Creating model with name: xgboost-2026-01-25-21-02-48-064
INFO:sagemaker:Creating endpoint-config with name xgboost-2026-01-25-21-02-48-064
INFO:sagemaker:Creating endpoint with name xgboost-2026-01-25-21-02-48-064


------!

Great! You should see an indicator like this when the deployment finishes:

```
-----------------!
```
We can test your deployment like so:

In [19]:
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()
inputs = test.copy()
# Drop the target variable
inputs = inputs.drop(columns=inputs.columns[0])
x_pred = xgb_predictor.predict(inputs.sample(5).values).decode('utf-8')
x_pred

'0.6030303239822388,0.6030303239822388,0.6030303239822388,0.6030303239822388,0.6030303239822388'

All systems go! To finish up the exercise, we're going to provide you with a DefaultModelMonitor and a suggested baseline. Combine the `xgb_predictor` and the provided `my_monitor` to configure the monitoring schedule for _hourly_ monitoring.

In [20]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [21]:
my_monitor.suggest_baseline(
    baseline_dataset=f's3://{bucket}/data/train.csv',
    dataset_format=DatasetFormat.csv(header=False),
)

INFO:sagemaker:Creating processing-job with name baseline-suggestion-job-2026-01-25-21-06-20-536


.................2026-01-25 21:09:07.952737: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2026-01-25 21:09:07.952772: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2026-01-25 21:09:09.366955: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2026-01-25 21:09:09.366990: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2026-01-25 21:09:09.367016: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-0-181-234.ec2.internal): /proc/driver/nvidia/version does not exist
2026-01-25 21:09:09.367348: I t

Below, provide the monitoring schedule:

In [34]:
from sagemaker.model_monitor import CronExpressionGenerator

my_monitor.create_monitoring_schedule(
    monitor_schedule_name="my-monitoring-schedule",
    endpoint_input=xgb_predictor.endpoint_name,
    statistics=my_monitor.baseline_statistics(),
    constraints=my_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
)

ERROR:sagemaker.model_monitor.model_monitoring:It seems that this object was already used to create an Amazon Model Monitoring Schedule. To create another, first delete the existing one using my_monitor.delete_monitoring_schedule().


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│    1 from sagemaker.model_monitor import CronExpressionGenerator                                 │
│    2                                                                                             │
│ ❱  3 my_monitor.create_monitoring_schedule(                                                      │
│    4 │   monitor_schedule_name="my-monitoring-schedule",                                         │
│    5 │   endpoint_input=xgb_predictor.endpoint_name,                                             │
│    6 │   statistics=my_monitor.baseline_statistics(),                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.12/site-packages/sagemaker/model_moni │
│ tor/model_monitoring.py:1998 in create_monitoring_schedule                                       │
│                                                                                                  │
│   1995 │   │   │   │   "using my_monitor.delete_monitoring_schedule()."                          │
│   1996 │   │   │   )                                                                             │
│   1997 │   │   │   logger.error(message)                                                         │
│ ❱ 1998 │   │   │   raise ValueError(message)                                                     │
│   1999 │   │                                                                                     │
│   2000 │   │   if (batch_transform_input is not None) ^ (endpoint_input is None):                │
│   2001 │   │   │   message = (                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: It seems that this object was already used to create an Amazon Model Monitoring Schedule. To create 
another, first delete the existing one using my_monitor.delete_monitoring_schedule().

Great job! You can check that your schedule was created by selecting the `SageMaker components and registries` tab on the far left.

In this exercise you configured Model Monitor to watch a simple model. Next, we'll monitor the same deployment for explainability.

__REMINDER:__ Don't leave your model deployed overnight. If you aren't going to follow up with the Clarify exercise within a few hours, use the code below to remove your model:

In [24]:
monitors = xgb_predictor.list_monitors()
for monitor in monitors:
    monitor.delete_monitoring_schedule()

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Deleting Monitoring Schedule with name: my-monitoring-schedule
INFO:sagemaker.model_monitor.model_monitoring:Deleting Data Quality Job Definition with name: data-quality-job-definition-2026-01-25-21-11-44-582


In [25]:
xgb_predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: xgboost-2026-01-25-21-02-48-064
INFO:sagemaker:Deleting endpoint with name: xgboost-2026-01-25-21-02-48-064


## Clarify

For the last exercise we'll deploy an explainability monitor using Clarify. We're going to use the model that you deployed in the last exercise, but if you cleaned up your deployments from the previous exercise, that's ok! You can rerun the deployment from the previous exercise up to the point where we deployed our model. It'll look like this:

```python
xgb_predictor = model.deploy(
    initial_instance_count=1, instance_type='ml.m4.xlarge',
    data_capture_config=data_capture_config
)
```

Once your model is deployed, you can come back here. _REMINDER_: you need to clean up your deployment, don't leave it running overnight. We'll provide some code at the end to delete your deployment.

## Prep

We'll begin by reloading our data from the previous exercise.

In [26]:
data = datasets.load_wine()
df = pd.DataFrame(data['data'])
df.columns = data['feature_names']
df.rename(columns = {'od280/od315_of_diluted_wines':'od280_od315_of_diluted_wines'}, inplace=True)

We also need to put the target variable in the first column per the docs for our chosen algorithm: https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html

In [27]:
df["TARGET"] = data['target']
df.set_index(df.pop('TARGET'), inplace=True)
df.reset_index(inplace=True)

Now we'll upload the data to S3 as train and validation data:

In [28]:
delimiter = int(len(df)/2)
train, test = df.iloc[delimiter:], df.iloc[:delimiter]

train.to_csv("train.csv", header=False, index=False)
test.to_csv("validation.csv", header=False, index=False)

val_location = session.upload_data('./validation.csv', key_prefix="data")
train_location = session.upload_data('./train.csv', key_prefix="data")

s3_input_train = sagemaker.inputs.TrainingInput(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data=val_location, content_type='csv')

Great! Our data is staged and our model is deployed - let's monitor it for explainability. We need to define three config objects, the `SHAPConfig`, the `ModelConfig`, and the `ExplainabilityAnalysisConfig`. Below, we provide the `SHAPConfig`.

In [29]:
shap_config = sagemaker.clarify.SHAPConfig(
    baseline=[train.mean().astype(int).to_list()[1:]],
    num_samples=int(train.size),
    agg_method="mean_abs",
    save_local_shap_values=False,
)

Next up, fill in the blanks to define the `ModelConfig` and `ExplainabilityAnalysisConfig`.

In [30]:
model_config = sagemaker.clarify.ModelConfig(
    model_name=xgb_predictor.endpoint_name,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    content_type='text/csv',
    accept_type='text/csv'
)

analysis_config = sagemaker.model_monitor.ExplainabilityAnalysisConfig(
    explainability_config=shap_config,
    model_config=model_config,
    headers=train.columns.to_list()[1:]
    )

Before we apply our config, we need to create the monitor object. This is what we'll apply all our config to.

In [31]:
model_explainability_monitor = sagemaker.model_monitor.ModelExplainabilityMonitor(
    role=role,
    sagemaker_session=session,
    max_runtime_in_seconds=1800,
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


Everything's ready! Below, create a monitoring schedule using the configs we created. Set the schedule to run _daily_.

In [32]:
print(xgb_predictor.endpoint_name)

xgboost-2026-01-25-21-02-48-064


In [33]:
from sagemaker.model_monitor import CronExpressionGenerator


explainability_uri = f"s3://{bucket}/model_explainability"
model_explainability_monitor.create_monitoring_schedule(
    output_s3_uri=explainability_uri,
    analysis_config=analysis_config,
    endpoint_input=xgb_predictor.endpoint_name,
    schedule_cron_expression=CronExpressionGenerator.daily(),
)

INFO:sagemaker.model_monitor.clarify_model_monitoring:Uploading analysis config to {s3_uri}.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│    2                                                                                             │
│    3                                                                                             │
│    4 explainability_uri = f"s3://{bucket}/model_explainability"                                  │
│ ❱  5 model_explainability_monitor.create_monitoring_schedule(                                    │
│    6 │   output_s3_uri=explainability_uri,                                                       │
│    7 │   analysis_config=analysis_config,                                                        │
│    8 │   endpoint_input=xgb_predictor.endpoint_name,                                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.12/site-packages/sagemaker/model_moni │
│ tor/clarify_model_monitoring.py:1096 in create_monitoring_schedule                               │
│                                                                                                  │
│   1093 │   │   │   network_config=self.network_config,                                           │
│   1094 │   │   │   batch_transform_input=batch_transform_input,                                  │
│   1095 │   │   )                                                                                 │
│ ❱ 1096 │   │   self.sagemaker_session.sagemaker_client.create_model_explainability_job_definiti  │
│   1097 │   │   │   **request_dict                                                                │
│   1098 │   │   )                                                                                 │
│   1099                                                                                           │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.12/site-packages/botocore/client.py:6 │
│ 02 in _api_call                                                                                  │
│                                                                                                  │
│    599 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    600 │   │   │   │   )                                                                         │
│    601 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  602 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    603 │   │                                                                                     │
│    604 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    605                                                                                           │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.12/site-packages/botocore/context.py: │
│ 123 in wrapper                                                                                   │
│                                                                                                  │
│   120 │   │   │   with start_as_current_context():                                               │
│   121 │   │   │   │   if hook:                                                                   │
│   122 │   │   │   │   │   hook()                                                                 │
│ ❱ 123 │   │   │   │   return func(*args, **kwargs)         

Way to go! You can check that your schedule was created by selecting the `SageMaker components and registries` tab on the far left.

In this exercise you deployed a monitor for explainability to your SageMaker endpoint. This is the last exercise - you'll apply these learnings again in your Project at the end of the course.



__REMINDER:__ Don't leave your model deployed overnight. Use the code below to remove your model:

In [ ]:
monitors = xgb_predictor.list_monitors()
for monitor in monitors:
    monitor.delete_monitoring_schedule()

In [ ]:
xgb_predictor.delete_endpoint()